## Eduardo Aydar de Oliveira - Everis
## Edgar Socrates Caria - Everis



---



# **``Cálculo das estimativas das notas - ENEM  ``**


### 1. Carregamento dos Dados
Download dos arquivos de dados das escolas, e carga no Python.

---


**1.1 Importação das Lib's necessárias**

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.2.1/spark-2.2.1-bin-hadoop2.7.tgz
!tar xf spark-2.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"

In [ ]:
!pip install -U -q PyDrive

In [ ]:
# pydrive documentation: https://pythonhosted.org/PyDrive/filelist.html

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1i-DzRlw6LzCttSt5d1QrPIlHyMR1td4V/view?usp=sharing

 # add files to Collaboratory
escolas_list = '1i-DzRlw6LzCttSt5d1QrPIlHyMR1td4V'
escolas_list = drive.CreateFile({'id': escolas_list})
escolas_list.GetContentFile('ENEM2015.csv')

# https://drive.google.com/file/d/1U9eBMS6bsDWCHxpqXQ6DZQOtwrCnuVeV/view?usp=sharing

 # add files to Collaboratory
escolas_enem = '1U9eBMS6bsDWCHxpqXQ6DZQOtwrCnuVeV'
escolas_enem = drive.CreateFile({'id': escolas_enem})
escolas_enem.GetContentFile('ESC2013_RMSP_CEM.csv')


In [ ]:
!ls

datalab			   spark-2.2.1-bin-hadoop2.7.tgz
ENEM2015.csv		   spark-2.2.1-bin-hadoop2.7.tgz.1
ESC2013_RMSP_CEM.csv	   spark-2.2.1-bin-hadoop2.7.tgz.2
spark-2.2.1-bin-hadoop2.7


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from operator import add
from pyspark import SparkContext

### 1.2 Carregamento dos Dados das escolas

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import math

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

In [ ]:
sc = SparkContext.getOrCreate()

escolas = spark.read.csv('ENEM2015.csv', inferSchema=True, header=True)
type(escolas)
escolas.take(10)


[Row(CODIGO DA ENTIDADE=35000139, NUMERO DE PARTICIPANTES=142, Classificacao=1),
 Row(CODIGO DA ENTIDADE=35000140, NUMERO DE PARTICIPANTES=79, Classificacao=1),
 Row(CODIGO DA ENTIDADE=35000267, NUMERO DE PARTICIPANTES=131, Classificacao=2),
 Row(CODIGO DA ENTIDADE=35000280, NUMERO DE PARTICIPANTES=146, Classificacao=1),
 Row(CODIGO DA ENTIDADE=35000334, NUMERO DE PARTICIPANTES=95, Classificacao=1),
 Row(CODIGO DA ENTIDADE=35000413, NUMERO DE PARTICIPANTES=100, Classificacao=1),
 Row(CODIGO DA ENTIDADE=35000486, NUMERO DE PARTICIPANTES=146, Classificacao=3),
 Row(CODIGO DA ENTIDADE=35000553, NUMERO DE PARTICIPANTES=172, Classificacao=2),
 Row(CODIGO DA ENTIDADE=35000620, NUMERO DE PARTICIPANTES=81, Classificacao=2),
 Row(CODIGO DA ENTIDADE=35000668, NUMERO DE PARTICIPANTES=44, Classificacao=1)]

In [ ]:
lista = spark.read.csv('ESC2013_RMSP_CEM.csv', inferSchema=True, header=True)
type(lista)
lista.take(10)

[Row(ID=90, LONGITUDE=-47143986, LATITUDE=-23986886, CODESC=35080548, CODESCTX=35080548, NOMEESC=u'BAIRRO DOS BARNABES RECANTO DAS ORQUIDEAS', NOMEMUN=u'JUQUITIBA', CODMUN=3526209, NOMDIST=u'BARNABES', CODDIST=352620910.0, COD_SC=None, COD_DEP=2, TIP_DEP=u'Estadual', BAIRRO=u'BAIRRO DOS BARNABES', CEP=6950000, END_ESC=u'RUA ANTONIO FRANCISCO PEREIRA', NUM_ESC=u'285', DDD=u'11', TELEFONE=u'46841502', LOCALIZA=1, LAB_INF=0, QD_ESP_C=0, QD_ESP_D=1, BIBLIO=0, ENS_REG=1, REG_IN_CR=0, REG_IN_PE=0, REG_FU_8A=0, REG_FU_9A=1, REG_MED=0, REG_MED_I=0, REG_MED_N=0, REG_MED_P=0, ENS_ESP=0, ESP_IN_CR=0, ESP_IN_PE=0, ESP_FU_8A=0, ESP_FU_9A=0, ESP_MED=0, ESP_MED_I=0, ESP_MED_N=0, ESP_MED_P=0, ESP_EJA_F=0, ESP_EJA_M=0, EJA=0, EJA_FUND=0, EJA_MED=0, EJA_PROJ=0, AP5EF_07=93.6, AP9EF_07=nan, AP3EM_07=nan, AB1EM_07=nan, AP5EF_08=97.5, AP9EF_08=nan, AP3EM_08=nan, AB1EM_08=nan, AP5EF_09=94.4, AP9EF_09=nan, AP3EM_09=nan, AB1EM_09=nan, AP5EF_10=98.4, AP9EF_10=nan, AP3EM_10=nan, AB1EM_10=nan, AP5EF_11=96.2, AP9

### 2. Seleção das Colunas do Dataset


Utilizado o código da escola e a classificação em 2015,  para comparação com o modelo estimado.

In [ ]:
esc_df = escolas.select("CODIGO DA ENTIDADE", "Classificacao")
esc_df = esc_df.toDF("COD", "Classificacao")
esc_df.take(10)

[Row(COD=35000139, Classificacao=1),
 Row(COD=35000140, Classificacao=1),
 Row(COD=35000267, Classificacao=2),
 Row(COD=35000280, Classificacao=1),
 Row(COD=35000334, Classificacao=1),
 Row(COD=35000413, Classificacao=1),
 Row(COD=35000486, Classificacao=3),
 Row(COD=35000553, Classificacao=2),
 Row(COD=35000620, Classificacao=2),
 Row(COD=35000668, Classificacao=1)]

Utilizadas as médias das notas anuais das escolas (2007, 2009, 2011, 2013) para determinação da média estimada para 2015.

In [ ]:
list_df = lista.select("CODESC", "NP07_AF", "NP09_AF", "NP11_AF", "NP13_AF")
list_df = list_df.toDF("COD", "NP07_AF", "NP09_AF", "NP11_AF", "NP13_AF")
list_df.take(10)

[Row(COD=35080548, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan),
 Row(COD=35352524, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan),
 Row(COD=35069929, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan),
 Row(COD=35036365, NP07_AF=4.18, NP09_AF=4.33, NP11_AF=4.85, NP13_AF=4.76),
 Row(COD=35913424, NP07_AF=4.47, NP09_AF=2.39, NP11_AF=4.9, NP13_AF=4.61),
 Row(COD=35915282, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan),
 Row(COD=35910569, NP07_AF=4.12, NP09_AF=nan, NP11_AF=4.88, NP13_AF=5.05),
 Row(COD=35247364, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan),
 Row(COD=35010157, NP07_AF=4.58, NP09_AF=4.81, NP11_AF=4.65, NP13_AF=4.76),
 Row(COD=35352536, NP07_AF=nan, NP09_AF=nan, NP11_AF=nan, NP13_AF=nan)]

### 2.2 Limpeza dos Dados

### 2.2.1 Identificação de Dados Duplicados


In [ ]:
esc_df.count() - esc_df.dropDuplicates().count()

0

In [ ]:
list_df.count() - list_df.dropDuplicates().count()

0

### 2.2.2 Identificação de Dados faltantes

In [ ]:
esc_df.count() - esc_df.dropDuplicates().dropna(how="any", subset=["COD", "Classificacao"]).count()

0

In [ ]:
list_df.count() - list_df.dropDuplicates().dropna(how="any", subset=["NP07_AF", "NP09_AF", "NP11_AF", "NP13_AF"]).count()

9087

### 2.3 Preenchimento dos dados faltantes

In [ ]:
esc_df = esc_df.dropDuplicates().fillna(value=0, subset=["CODIGO DA ENTIDADE", "Classificacao"])
esc_df.count()

1243

In [ ]:
list_df = list_df.dropDuplicates().fillna(value=0, subset=["NP07_AF", "NP09_AF", "NP11_AF", "NP13_AF"])
list_df.count()

10937

### JOIN dos Dados
Efetuado o join dos dados de 2015 com os dados anuais, utilizando-se como chave o código da escola.

In [ ]:
comb = esc_df.join(list_df, "COD", "left")

comb.show()

+--------+-------------+-------+-------+-------+-------+
|     COD|Classificacao|NP07_AF|NP09_AF|NP11_AF|NP13_AF|
+--------+-------------+-------+-------+-------+-------+
|35000930|            2|   4.73|   5.01|    5.2|   4.96|
|35007882|            1|   4.39|   4.47|   4.26|   4.69|
|35008308|            1|   5.36|    5.4|   5.11|   4.88|
|35009465|            2|   5.26|    5.5|   5.31|    5.6|
|35010297|            1|   4.61|   4.86|    4.7|   5.04|
|35120285|            4|    0.0|    0.0|    0.0|    0.0|
|35187537|            2|    0.0|    0.0|    0.0|    0.0|
|35809559|            2|    0.0|    0.0|    0.0|    0.0|
|35105041|            3|    0.0|    0.0|    0.0|    0.0|
|35124291|            2|    0.0|    0.0|    0.0|    0.0|
|35155019|            2|    0.0|    0.0|    0.0|    0.0|
|35185449|            2|    0.0|    0.0|    0.0|    0.0|
|35477904|            2|    0.0|    0.0|    0.0|   6.23|
|35809305|            2|    0.0|    0.0|    0.0|    0.0|
|35038428|            1|   4.12

In [ ]:
comb

DataFrame[COD: int, Classificacao: int, NP07_AF: double, NP09_AF: double, NP11_AF: double, NP13_AF: double]

In [ ]:
comb.describe()

DataFrame[summary: string, COD: string, Classificacao: string, NP07_AF: string, NP09_AF: string, NP11_AF: string, NP13_AF: string]

In [ ]:
comb.count()

1243

## Criação de um Modelo de Classificação

### Montagem dos dados de Treinamento / Teste
30% = Training Data


70% = Test Data

In [ ]:
splits = comb.randomSplit([0.3, 0.7])
treino = splits[0]
teste = splits[1]
treino_rows = treino.count()
teste_rows = teste.count()
print "Treino Rows:", treino_rows, " Teste Rows:", teste_rows

Treino Rows: 349  Teste Rows: 894


In [ ]:
treino

DataFrame[COD: int, Classificacao: int, NP07_AF: double, NP09_AF: double, NP11_AF: double, NP13_AF: double]

### Preparação dos Dados de Treinamento
Utilização de classe  **VectorAssembler** para transformar as colunas das médias anuais em um vetor, e renomear a coluna "Classificação".

In [ ]:
assembler = VectorAssembler(inputCols = ["NP07_AF", "NP09_AF", "NP11_AF", "NP13_AF"], outputCol="hist")
training = assembler.transform(treino).select(col("hist"), (col("Classificacao").cast("Int").alias("predicao")))
training.show()

+--------------------+--------+
|                hist|predicao|
+--------------------+--------+
|[4.61,4.86,4.7,5.04]|       1|
|           (4,[],[])|       4|
|           (4,[],[])|       3|
|  [4.7,4.72,5.1,3.8]|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       3|
|           (4,[],[])|       2|
|[4.82,4.59,5.31,5...|       2|
|[5.07,5.05,5.07,0.0]|       1|
|[4.69,4.78,5.17,5.2]|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
| [4.82,4.51,5.2,5.0]|       2|
|[4.94,5.02,5.0,4.73]|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
|           (4,[],[])|       2|
+--------------------+--------+
only showing top 20 rows



### Treino de um Modelo de Classificação
Utilizada a Regressão Logística.

In [ ]:
lr = LogisticRegression(labelCol="predicao",featuresCol="hist",maxIter=10,regParam=0.3)
model = lr.fit(training)
print "Model trained!"

Model trained!


### Preparando os Dados de Teste


In [ ]:
teste_mod = assembler.transform(teste).select(col("hist"), (col("Classificacao")).cast("Int").alias("real"))
teste_mod.show()

+--------------------+----+
|                hist|real|
+--------------------+----+
|[4.73,5.01,5.2,4.96]|   2|
|[4.39,4.47,4.26,4...|   1|
|[5.36,5.4,5.11,4.88]|   1|
| [5.26,5.5,5.31,5.6]|   2|
|           (4,[],[])|   2|
|           (4,[],[])|   2|
|           (4,[],[])|   2|
|           (4,[],[])|   2|
|           (4,[],[])|   2|
|      (4,[3],[6.23])|   2|
|           (4,[],[])|   2|
|[4.12,4.22,4.32,4...|   1|
|           (4,[],[])|   3|
|           (4,[],[])|   2|
|           (4,[],[])|   3|
|           (4,[],[])|   2|
|[4.66,4.78,4.92,4...|   1|
|           (4,[],[])|   4|
|           (4,[],[])|   2|
|           (4,[],[])|   2|
+--------------------+----+
only showing top 20 rows



### Teste do Modelo


In [ ]:
testando = model.transform(teste_mod)
estimado = testando.select("hist", "prediction", "real")
estimado.show(100)

+--------------------+----------+----+
|                hist|prediction|real|
+--------------------+----------+----+
|[4.73,5.01,5.2,4.96]|       2.0|   2|
|[4.39,4.47,4.26,4...|       2.0|   1|
|[5.36,5.4,5.11,4.88]|       1.0|   1|
| [5.26,5.5,5.31,5.6]|       1.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|      (4,[3],[6.23])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|[4.12,4.22,4.32,4...|       2.0|   1|
|           (4,[],[])|       2.0|   3|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   3|
|           (4,[],[])|       2.0|   2|
|[4.66,4.78,4.92,4...|       2.0|   1|
|           (4,[],[])|       2.0|   4|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|       2.0|   2|
|           (4,[],[])|   

In [ ]:
type(estimado)
                              

pyspark.sql.dataframe.DataFrame

In [ ]:
est_rd = estimado.select(round("prediction", 0).alias("Estimativa"), "real")
est_rd.show()
est_rd.count()

+----------+----+
|Estimativa|real|
+----------+----+
|       2.0|   2|
|       2.0|   1|
|       1.0|   1|
|       1.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   2|
|       2.0|   1|
|       2.0|   3|
|       2.0|   2|
|       2.0|   3|
|       2.0|   2|
|       2.0|   1|
|       2.0|   4|
|       2.0|   2|
|       2.0|   2|
+----------+----+
only showing top 20 rows



894

In [ ]:
resultado = est_rd.withColumn("res", when (est_rd.Estimativa == est_rd.real, True).otherwise(False))
resultado.show()
resultado.count()


+----------+----+-----+
|Estimativa|real|  res|
+----------+----+-----+
|       2.0|   2| true|
|       2.0|   1|false|
|       1.0|   1| true|
|       1.0|   2|false|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   2| true|
|       2.0|   1|false|
|       2.0|   3|false|
|       2.0|   2| true|
|       2.0|   3|false|
|       2.0|   2| true|
|       2.0|   1|false|
|       2.0|   4|false|
|       2.0|   2| true|
|       2.0|   2| true|
+----------+----+-----+
only showing top 20 rows



894

In [ ]:
cal = resultado.where(col("res") == False)
cal.count()

298

### Comparação Final

In [ ]:
((894-298)*100)/894

66

### Conclusão: O percentual de acerto, de acordo com os cálculos, foi de 66%. Acreditamos que poderiam ter sido usadas mais variáveis e refinamento nos cálculos para aperfeiçoar o resultado, além de uma simulação com mais interações.

### Att,
# Eduardo / Edgar